 ## Puzle de 8 con RL
 
Se aplica el algoritmo Q-learning para diseñar un agente que aprenda a resolver el puzzle del 8. 
 

Para poder aplicar Q-learning necesitamos numerar las diferentes acciones y estados para poder crear la q_table.

In [84]:
import numpy as np
import random
import math
from IPython.display import clear_output

In [85]:
from search import Problem
import itertools
class Ocho_Puzzle(Problem):
    """Problema a del 8-puzzle.  Los estados serán tuplas de nueve elementos,
    permutaciones de los números del 0 al 8 (el 0 es el hueco). Representan la
    disposición de las fichas en el tablero, leídas por filas de arriba a
    abajo, y dentro de cada fila, de izquierda a derecha. Las cuatro
    acciones del problema las representaremos mediante las cadenas:
    "Mover hueco arriba", "Mover hueco abajo", "Mover hueco izquierda" y
    "Mover hueco derecha", respectivamente."""""

    def __init__(self, initial, goal=(1, 2, 3, 4, 5, 6, 7, 8, 0)):
        """ Define goal state and initialize a problem """
        self.goal = goal
        Problem.__init__(self, initial, goal)
        
    
    def actions(self,estado):
     
        pos_hueco=estado.index(0) # busco la posicion del 0
        accs=list()
        if pos_hueco not in (0,1,2):
            accs.append("Mover hueco arriba")
        if pos_hueco not in (2,5,8):
            accs.append("Mover hueco derecha")
        if pos_hueco not in (0,3,6): 
            accs.append("Mover hueco izquierda") 
        if pos_hueco not in (6,7,8):
            accs.append("Mover hueco abajo")
                
        return accs     

    def result(self,estado,accion):
        pos_hueco = estado.index(0)
        l = list(estado)
        if accion == "Mover hueco arriba":
            l[pos_hueco] = l[pos_hueco-3]
            l[pos_hueco-3] = 0
        if accion == "Mover hueco abajo":
            l[pos_hueco] = l[pos_hueco + 3]
            l[pos_hueco + 3] = 0
        if accion == "Mover hueco derecha":
            l[pos_hueco] = l[pos_hueco+1]
            l[pos_hueco+1] = 0
        if accion == "Mover hueco izquierda":
            l[pos_hueco] = l[pos_hueco-1]
            l[pos_hueco-1] = 0
        return tuple(l)
    
    def h(self, node):
        """ Return the heuristic value for a given state. """
        return 1
    
    def check_solvability(self, state):
        """ Checks if the given state is solvable """
    # The solvability of a configuration can be checked by calculating the Inversion Permutation. 
    #If the total Inversion Permutation is even then the initial configuration is solvable else the initial configuration is not solvable which means that only 9!/2 initial states lead to a solution.
        inversion = 0
        for i in range(len(state)):
            for j in range(i+1, len(state)):
                if (state[i] > state[j]) and state[i] != 0 and state[j]!= 0:
                    inversion += 1
        return inversion % 2 == 0
    
    def lista_estados(self,estado):
        i = 0;
        diccionario = {}
        for e in itertools.permutations(estado):
            diccionario[e] = i
            i = i + 1;
        return diccionario  
    
    def listaAcciones(self):
        listaAcciones = {}
        listaAcciones[0] ="Mover hueco arriba" 
        listaAcciones[1] ="Mover hueco abajo"
        listaAcciones[2] ="Mover hueco derecha"
        listaAcciones[3] ="Mover hueco izquierda"
        return listaAcciones
    
    def listaAcciones2(self):
        listaAcciones2 = {}
        listaAcciones2["Mover hueco arriba"] = 0
        listaAcciones2["Mover hueco abajo"] =1
        listaAcciones2["Mover hueco derecha"] =2
        listaAcciones2["Mover hueco izquierda"] =3
        return listaAcciones2

In [86]:
# creamos e inicializamos la Q-Table
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])    

In [91]:
initial=(1,2,0,4,5,3,7,8,6)

In [92]:
ocho = Ocho_Puzzle(initial)
print("estado objetivo",ocho.goal)   
ocho.__init__(initial) 
state = ocho.initial


estado objetivo (1, 2, 3, 4, 5, 6, 7, 8, 0)


In [93]:
def recompensa(state):
    if state==ocho.goal:
        return 100
    else:
        return 0

In [94]:
def agente_puzle8(episodios=10000, alpha = 0.7, gamma = 0.6, epsilon = 0.8):
    """Creamos y entrenamos el agente con los parametros dados"""
    state = ocho.initial
    listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    exitos=0
    for i in range(1, episodios):
        if random.uniform(0, 1) < epsilon:
            mejor = listaAcciones2[random.choice(ocho.actions(state))]
            #print("La accion aleatoria seleccionada es ", listaAcciones[mejor])
        else:
            mejor = 0
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor]:
                    mejor = accion
                    #print("La mejor accion es ", listaAcciones[mejor])

        if (listaAcciones[mejor] in ocho.actions(state)):
            sig_estado = ocho.result(state , listaAcciones[mejor])
            reward = recompensa(sig_estado)
        else:
            reward = -50 #accion no aplicable
            sig_estado =state
        old_value = q_table[listaEstados[state]][mejor]
        next_max = np.max(q_table[listaEstados[sig_estado]][mejor])
        new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
        q_table[listaEstados[state], mejor] = new_value
        state = sig_estado
        if state == ocho.goal:
            #Termino el episodio con éxito. Se ha encontrado la solución
            state = ocho.initial
            exitos=exitos+1
            continue
        if i % 100 == 0:
            clear_output(wait=True)
            print(f"Episode: {i}")

    print("Training finished.\n") 
    print("La Q-table:\n") 
    np.set_printoptions(threshold=np.inf)
    print(q_table)
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos: ", exitos)

In [ ]:
# MODIFICA LOS PARÁMETROS Y OBSERVA EL APRENDIZAJE
alpha = 0.5
gamma = 0.9
epsilon = 0.8
episodios=1000000

In [ ]:
%%time
# puedes medir el tiempo de aprendizaje 

agente_puzle8(episodios,alpha,gamma,epsilon)
# Generar el mapeo de estados a índices
lista_estados = ocho.lista_estados(initial)

# Mostrar el mapeo completo
for estado, indice in lista_estados.items():
    print(f"Índice en Q-table: {indice} -> Estado: {estado}")

In [ ]:
def evalua_agente_puzle8(initial):
    """Solo explotación del agente sin modificar la tabla Q"""
    exitos=0
    fallos=0
    state = initial
    episodios =100
    #listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(state)
    for i in range(1, episodios+1):
        done =False
        while not done:
            mejor_accion = 0        
            for accion in listaAcciones.keys():
                if q_table[listaEstados[state]][accion] > q_table[listaEstados[state]][mejor_accion]:
                    mejor_accion = accion                
            print("La accion aleatoria seleccionada es ", listaAcciones[mejor_accion])
            print("Estado:", state)
            if (listaAcciones[mejor_accion] in ocho.actions(state)):
                state = ocho.result(state , listaAcciones[mejor_accion])
                print("Estado siguiente:", state)
            else: 
                #termino con fallo porque la accion no es aplicable
                fallos=fallos+1
                done = True            
            if state == ocho.goal:
                #print("Termino el episodio con éxito. Se ha encontrado la solución")
                exitos=exitos+1 
                done =True 
                state=initial
            if i % 100 == 0:
                clear_output(wait=True)
                print(f"Episodio: {i}")
        
    print("Evaluación terminada.\n") 
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos:", exitos)
    print("Número de éxitos:", (exitos/episodios)*100, " %")  
    

In [ ]:
estado = (1,2,3,4,5,0,7,8,6)
#estado = (1,2,3,4,5,6,7,0,8)
evalua_agente_puzle8(estado)

¿Funciona bien el entrenamiento?
¿Puedes mejorarlo?

El agente ha aprendido con QLearning a resolver el problema del 8 puzle empezando en un estado inicial fijo y ha tardado un cierto tiempo.
Una vez entrenado puedes ver qué tal se comporta (usando sólo la parte de explotación de la matriz Q aprendida y midiendo el porcentaje de éxitos). 

#### Responde a las siguientes cuestiones:

- Justifica de forma razonada si esta aproximación con aprendizaje es mejor que la aproximación con búsqueda heurística de la primera práctica.
- Observa la matriz de aprendizaje ¿qué ha aprendido el agente?
- Si entrenas el agente para un estado inicial ¿cómo se comporta al evaluarlo con otro estado inicial? 
- Realiza los cambios necesarios para que el agente aprenda a resolver el problema desde cualquier estado inicial.
- Te parece adecuada la función de recompensa utilizada. ¿Crees que se podría mejorar el rendimiento del agente cambiando la recompensa?
- Comprueba cómo se comporta el aprendizaje con distintas configuraciones de los parámetros. Indica claramente cuál es la mejor configuración que has encontrado.

#### Justifica de forma razonada si esta aproximación con aprendizaje es mejor que la aproximación con búsqueda heurística de la primera práctica.

Ni idea honestamente.

#### Observa la matriz de aprendizaje ¿qué ha aprendido el agente?

Para el análisis de la matriz de aprendizaje primero debemos saber cómo está representada:
- Las **columnas** hacen referencia a las acciones que pueden realizarse desde cierto estado (mover hueco arriba, abajo, derecha e izquierda).
- Las **filas** representan los distintos estados posibles del puzle (todas las permutaciones de piezas).

Se usa la línea [np.set_printoptions(threshold=np.inf)] en la función agente_puzle8() para visualizar la QTable entera y se imprimen los distintos estados del puzle con su índice en la matriz con una lista y un bucle for.
Los parámetros se usan con los valores : alpha = 0.5, gamma = 0.9 y epsilon = 0.8. Tras 1000000 episodios de aprendizaje y un tiempo real de ejecución de 1 minutos y 15 segundos, se obtiene una QTable en la que observamos:

- Valores negativos: Hacen referencia a acciones que no pueden darse como mover el hueco arriba si está ya en una de las posiciones de la primera fila del tablero. Además, al tener un valor alto de gamma, se consigue que los valores se propaguen mucho y allá distintos valores negativos en la matriz.

- Valores positivos: Están asociados a movimiento que llevan al estado objetivo y, al igual que pasaba con los valores neagtivos, hay muy variados por la gran importancia del largo plazo. Un ejemplo muy claro es la solución desde el estado inicial de mover dos veces el hueco hacia abajo: el primer movimiento hacia abajo tiene una recompensa de 89.99982024 (fila 1, columna 2) y el segundo una recompensa de 99.99980927 (fila 2304, columna 2).

- Valores 0: Es posible que vengan dados por la gran cantidad de permutaciones posibles y que, a pesar del valor de epsilon de 0.5, no todos los estados sean explorados depués de el número de episodios dado.

#### Si entrenas el agente para un estado inicial ¿cómo se comporta al evaluarlo con otro estado inicial? 

Tal y como se daban los valores de los parámetros anteriormente, realmente depende del estado elegido:
- Si elegimos un estado que forme parte de la solución del estado inicial (1,2,3,4,5,0,7,8,6) donde se ha dado el primer movimiento hacia abajo, el agente es capaz de encontrar la solución.
- Si elegimos un estado del tablero completamente diferente (1,2,3,4,5,6,7,0,8), el agente no es capaz de resolver el problema. Esto se debe a que la QTable queda principalmente llena de valores 0 pues es muy grande y no se exploran correctamente todas las posibilidades. 

En la mayor parte de los casos, dado un estado inicial aleatorio y esta matriz basada en el estado inicial anterior, el agente no podría encontrar una solución.

#### Realiza los cambios necesarios para que el agente aprenda a resolver el problema desde cualquier estado inicial.

In [ ]:
def agente_puzle8(episodios=10000, alpha=0.7, gamma=0.6, epsilon=0.8):
    listaAcciones2 = ocho.listaAcciones2()
    listaAcciones = ocho.listaAcciones()
    listaEstados = ocho.lista_estados(ocho.initial)
    exitos = 0
    
    for i in range(1, episodios):
        # Seleccionar un estado inicial aleatorio y comprobando que sea resoluble
        estado = list(random.sample(range(9), 9))
        while not ocho.check_solvability(estado):
            estado = list(random.sample(range(9), 9))
        estado = tuple(estado)
        
        # Realizar el episodio
        for i in range(1, episodios):
            if random.uniform(0, 1) < epsilon:
                mejor = listaAcciones2[random.choice(ocho.actions(estado))]
                #print("La accion aleatoria seleccionada es ", listaAcciones[mejor])
            else:
                mejor = 0
                for accion in listaAcciones.keys():
                    if q_table[listaEstados[estado]][accion] > q_table[listaEstados[estado]][mejor]:
                        mejor = accion
                        #print("La mejor accion es ", listaAcciones[mejor])

            if (listaAcciones[mejor] in ocho.actions(estado)):
                sig_estado = ocho.result(estado , listaAcciones[mejor])
                reward = recompensa(sig_estado)
            else:
                reward = -50 #accion no aplicable
                sig_estado =estado
            
            old_value = q_table[listaEstados[estado]][mejor]
            next_max = np.max(q_table[listaEstados[sig_estado]][mejor])
            new_value = (1 - alpha) * old_value + alpha * (reward + gamma * next_max)
            q_table[listaEstados[estado], mejor] = new_value
            estado = sig_estado
            if estado == ocho.goal:
                #Termino el episodio con éxito. Se ha encontrado la solución
                estado = ocho.initial
                exitos=exitos+1
                continue
            if i % 100 == 0:
                clear_output(wait=True)
                print(f"Episode: {i}")
            
            # Contador de éxitos
            exitos += 1
    

    print("Training finished.\n") 
    print("La Q-table:\n") 
    print(q_table)
    # Métrica del % de veces que ha llegado al estado objetivo. 
    print("Número de éxitos: ", exitos)

In [ ]:
alpha = 0.5
gamma = 0.9
epsilon = 0.5
episodios=500000

In [ ]:
initial = (1, 2, 0, 4, 5, 3, 7, 8, 6)
ocho = Ocho_Puzzle(initial)
totalAcciones = 4
totalEstados = math.factorial(9)
q_table = np.zeros([totalEstados, totalAcciones])    

In [ ]:
%%time
# puedes medir el tiempo de aprendizaje 
agente_puzle8(episodios, alpha, gamma, epsilon)

In [ ]:
#estado = (1,2,3,4,5,0,7,8,6)
estado = (1,2,3,4,5,6,7,0,8)
evalua_agente_puzle8(estado)

No lo consigo la verdad no entiendo encima tarda un huevoooo.

#### Te parece adecuada la función de recompensa utilizada. ¿Crees que se podría mejorar el rendimiento del agente cambiando la recompensa?

Con el objetivo de que encuentre la solución más óptima, se penaliza cada paso con un -1 consiguiendo que el agente minimice el número de acciones hasta la solución maximizando la recompensa.


In [ ]:
def recompensa(state):
    if state==ocho.goal:
        return 100
    else:
        return -1

#### Comprueba cómo se comporta el aprendizaje con distintas configuraciones de los parámetros. Indica claramente cuál es la mejor configuración que has encontrado.

Se hace un análisis de la modificación de los parámetros sobre las funciones dadas inicialmente:

In [95]:
q_table = np.zeros([totalEstados, totalAcciones])   
alpha = 0.5
gamma = 0.9
epsilon = 0.8
episodios=1000000

In [ ]:
%%time

agente_puzle8(episodios,alpha,gamma,epsilon)
lista_estados = ocho.lista_estados(initial)

for estado, indice in lista_estados.items():
    print(f"Índice en Q-table: {indice} -> Estado: {estado}")

- Primer caso (α = 0.5; γ = 0.9; ε=0.8):
- Segundo caso (α = 0.7; γ = 0.9; ε=0.8):
- Tercer caso (α = 0.5; γ = 0.5; ε=0.8):
- Cuarto caso (α = 0.7; γ = 0.5; ε=0.8):
- Quinto caso (α = 0.5; γ = 0.5; ε=0.5):
- Sexto caso (α = 0.5; γ = 0.5; ε=0.2):